In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pprint import pprint

In [3]:
from collections import Counter, defaultdict

import json
from xgboost import XGBClassifier
import sklearn.tree
import sklearn.metrics
from sklearn.model_selection import cross_val_score

In [19]:
from sample_chapters import *
from feature_extraction import *
from classify import *

In [23]:
def extract_texts_and_characters(annotated_data):
    full_characters = np.asarray([datum['character'] for datum in annotated_data])
    full_texts = np.asarray([datum['text'] for datum in annotated_data])
    return full_texts, full_characters

In [24]:
from sklearn.model_selection import KFold

def evaluate(annotated_data, classifier=XGBClassifier(), nicknames2name=dict(), n_splits=10):
    
    full_texts, full_characters = extract_texts_and_characters(annotated_data)
    
    scores = []
    for train_inds, test_inds in KFold(n_splits=n_splits).split(annotated_data):
        train_texts = full_texts[train_inds]
        train_characters = full_characters[train_inds]
        
        test_characters = full_characters[test_inds]
        test_texts = full_texts[test_inds]
        

        train_classifier(train_texts, train_characters, classifier, nicknames2name)
        score = test_classifier(test_texts, test_characters, classifier, nicknames2name)
        
        print(score)
        scores.append(score)
    return scores

In [17]:
nicknames2name_GoT = {
    "Dany":"Daenerys",
    "Ned" : "Eddard",
    "Robert" : "Eddard",
    "Sam" : "Samwell",
}
  
with open("../flat_data/asoif01-04.json","r") as fh:
    ann_GoT = np.asarray(json.load(fh))

In [18]:
scores = evaluate(ann_GoT, XGBClassifier(), nicknames2name_GoT, n_splits=2)
np.mean(scores)

0.7421875
0.828125


0.78515625

In [32]:
cls = train_classifier(*extract_texts_and_characters(ann_GoT), 
                       classifier=XGBClassifier(),
                       nicknames2name=nicknames2name_GoT)

output_characters = run_classifier(extract_texts_and_characters(ann_GoT)[0], 
                       classifier=cls,
                       nicknames2name=nicknames2name_GoT)
reference_characters = [datum['character'] for datum in ann_GoT]

print("acc: ", sklearn.metrics.accuracy_score(output_characters, reference_characters))

SyntaxError: EOL while scanning string literal (<ipython-input-32-34262f8b8ae9>, line 11)

In [41]:
from sklearn.externals import joblib
joblib.dump(cls, "trained_models/GoT-no-headings.pkl")


['trained_models/GoT-no-headings.pkl']

In [43]:
cls2 = joblib.load("trained_models/GoT-no-headings.pkl")
output_characters = run_classifier(extract_texts_and_characters(ann_GoT)[0], 
                       classifier=cls2,
                       nicknames2name=nicknames2name_GoT)

In [44]:
print("acc: ", sklearn.metrics.accuracy_score(output_characters, reference_characters))

acc:  1.0


## Feature importance stuff

In [24]:
_, _,vector_keys = get_feature_vectors(ann_GoT[1]['text'])
feature_weights = list(zip(cls.feature_importances_,vector_keys))
feature_weights.sort(reverse=True)
feature_weights

[(0.122, 'before_POS_was_percent_,'),
 (0.097999997, 'rank_percent'),
 (0.092, 'occur_percent'),
 (0.082000002, 'after_POS_was_percent_.'),
 (0.074000001, 'after_POS_was_VBD'),
 (0.071999997, 'after_POS_was_percent_VBD'),
 (0.068000004, 'before_POS_was_.'),
 (0.064000003, 'rank'),
 (0.048, 'occur_count'),
 (0.039999999, 'before_POS_was_,'),
 (0.035999998, 'last_occur_percent'),
 (0.029999999, 'before_POS_was_percent_JJ'),
 (0.026000001, 'after_POS_was_percent_VBZ'),
 (0.02, 'before_POS_was_NN'),
 (0.016000001, 'after_POS_was_MD'),
 (0.014, 'after_POS_was_percent_NN'),
 (0.012, 'after_POS_was_percent_,'),
 (0.0099999998, 'last_occur_position'),
 (0.0099999998, "before_POS_was_percent_''"),
 (0.0080000004, 'before_POS_was_percent_.'),
 (0.0080000004, 'after_POS_was_percent_CC'),
 (0.0060000001, 'before_POS_was_percent_WRB'),
 (0.0060000001, 'before_POS_was_percent_VBN'),
 (0.0060000001, 'before_POS_was_percent_VBD'),
 (0.0060000001, 'before_POS_was_percent_NNP'),
 (0.0040000002, 'before_

array([ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.006,  0.   ,
        0.   ,  0.   ,  0.   ,  0.064,  0.   ,  0.   ,  0.   ,  0.03 ,
        0.006,  0.   ,  0.   ,  0.   ,  0.008,  0.   ,  0.002,  0.   ,
        0.   ,  0.098,  0.004,  0.   ,  0.01 ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.004,  0.   ,  0.   ,  0.   ,
        0.04 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.014,  0.   ,  0.   ,  0.   ,  0.072,
        0.074,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.004,
        0.   ,  0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.016,  0.002,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.026,
        0.   ,  0.   ,  0.01 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.002,  0.   ,  0.   ,  0.   ,
      

In [ ]:

scores = evaluate(ann_GoT, nicknames2name_GoT, XGBClassifier(n_estimators=100))
np.mean(scores)

In [45]:
with open("../flat_data/Warbreaker.json","r") as fh:
    warbreaker = json.load(fh)

In [58]:
textt='\n\nPrologue\nIt’s funny,  Vasher thought,  how many things begin with me getting thrown into prison. \nThe guards laughed to one another, slamming the cell door shut with a clang.  Vasher stood and dusted himself off, rolling his shoulder and wincing.  While the bottom half of his cell door was solid wood, the top half was barred, and he could see the three guards open his large duffle and rifle through his possessions.\nOne of them noticed him watching.  The guard was an oversized beast of a man with a shaved head and a dirty uniform that barely retained the bright yellow and blue coloring of the T’Telir city guard.\nBright colors,  Vasher thought.  I’ll have to get used to those again.   In any other nation, the vibrant blues and yellows would have been ridiculous on soldiers.  This, however, was Hallandren: land of Returned Gods, Lifeless servants, BioChromatic research, and--of course--\ncolor.\nThe large guard sauntered up to the cell door, leaving his friends to amuse themselves with Vasher’s belongings.  “They say you’re pretty tough,” the man said, sizing up Vasher.\nVasher did not respond.\n“The bartender says you beat down some twenty men in the brawl.”  The guard rubbed his chin.  “You don’t look that tough to me.  Either way, you should have known better than to strike a priest.  The others, they’ll spend a night locked up.  You, though. . .you’ll hang.\nColorless fool.”\nSanderson/Warbreaker/8\nVasher turned away.  His cell was functional, if unoriginal.  A thin slit at the top of one wall let in light, the stone walls dripped with water and moss, and a pile of dirty straw decomposed in the corner.\n“You ignoring me?” the guard asked, stepping closer to the door.  The colors of his uniform brightened, as if he’d stepped into a stronger light.  The change was slight.  Vasher didn’t have much Breath remaining, and so his aura didn’t do much to the colors around him.\nThe guard didn’t notice the change in color--just as he hadn’t noticed back in the bar, when he and his buddies had picked Vasher up off the floor and thrown him in their cart.  Of course, the change was so slight to the unaided eye that it would have been nearly impossible to pick out.\n“Here, now,” said one of the men looking through Vasher’s duffle.  “What’s  this? ”\nVasher had always found it interesting that the men who watched dungeons tended to be as bad as, or worse than, the men they guarded.  Perhaps that was deliberate.  Society didn’t seem to care if such men were outside the cells or in them, so long as they were kept away from more honest men.\nAssuming that such a thing existed.\nFrom Vasher’s bag, a guard pulled free a long object wrapped in white linen.  The man whistled as he unwrapped the cloth, revealing a long, thin-bladed sword in a silver sheath.  The hilt was pure black.  “Who do you suppose he stole  this from?”\nThe lead guard eyed Vasher, likely wondering if Vasher were some kind of nobleman.\nThough Hallandren had no aristocracy, many neighboring kingdoms had their lords and ladies.\nYet what lord would wear a drab brown cloak, ripped in several places?  What lord would sport bruises from a bar fight, a half-grown beard, and boots worn from years of walking?  The guard turned away, apparently convinced that Vasher was no lord.\nSanderson/Warbreaker/9\nHe was right.  And he was wrong.\n“Let me see that,” the lead guard said, taking the sword.  He grunted, obviously surprised by its weight.  He turned it about, noting the clasp that tied sheath to hilt, keeping the blade from being drawn.  He undid the clasp.\nThe colors in the room deepened.  They didn’t grow brighter--not like the guard’s vest had when he approached Vasher.  Instead, they grew  stronger.  Darker.  Reds became maroon.\nYellows hardened to gold.  Blues approached navy.\n“Be careful, friend,” Vasher said softly, “that sword can be dangerous.”\nThe guard looked up.  All was still.  Then, the guard snorted and walked away from Vasher’s cell, still carrying the sword.  The other two followed, bearing Vasher’s duffle, entering the guard room at the end of the hallway.\nThe door thumped shut.  Vasher immediately knelt beside the patch of straw, selecting a handful of sturdy lengths.  He pulled threads from his cloak--it was beginning to fray at the bottom--and tied the straw into the shape of a small person, perhaps three inches high, with bushy arms and legs.  He plucked a hair from one of his eyebrows, set it against the straw figure’s head, then reached into his boot and pulled out a brilliant red scarf.\nThen Vasher Breathed.\nIt flowed out of him, puffing into the air, translucent yet radiant, like the color of oil on water in the sun.  Vasher felt it leave: BioChromatic Breath, scholars called it.  Most people just called it Breath.  Each person had one.  Or, at least, that was how it usually went.  One person, one Breath.\nVasher had around fifty Breaths, just enough to reach the First Heightening.  Having so few made him feel poor compared to once he’d once held, but many would consider fifty Breaths Sanderson/Warbreaker/10\nto be a great treasure.  Unfortunately, even Awakening a small figure made from organic material--using a piece of his own body as a focus--drained away some half of his Breaths.\nThe little straw figure jerked, sucking in the Breath.  In Vasher’s hand, half of the brilliant red scarf faded to grey.  Vasher leaned down--imagining what he wanted the figure to do--and completed the final step of the process as he gave the Command.\n“Fetch keys,” he said.\nThe straw figure stood and raised its single eyebrow toward Vasher.\nVasher pointed toward the guard room.  From it, he heard sudden shouts of surprise.\nNot much time,  he thought.\nThe straw person ran along the floor, then jumped up, vaulting between the bars.  Vasher pulled off his cloak and set it on the floor.  It was the perfect shape of a person--marked with rips that matched the scars on Vasher’s body, its hood cut with holes to match Vasher’s eyes.  The closer an object was to human shape and form, the fewer Breaths it took to Awaken.\nVasher leaned down, trying not to think of the days when he’d had enough Breaths to Awaken without regard for shape or focus.  That had been a different time.  Wincing, he pulled a tuft of hair from his head, then sprinkled it across the hood of the cloak.\nOnce again, he Breathed.\nIt took the rest of his Breath.  With it gone--the cloak trembling, the scarf losing the rest of its color--Vasher felt. . .dimmer.  Losing one’s Breath was not fatal.  Indeed, the extra Breaths Vasher used had once belonged to other people.  Vasher didn’t know who they were; he hadn’t gathered these Breaths himself.  They had been given to him.  But, of course, that was the way it was always supposed to work.  One could not take Breath by force.\nSanderson/Warbreaker/11\nBeing void of Breath  did change him.  Colors didn’t seem as bright.  He couldn’t  feel the bustling people moving about in the city above, a connection he normally took for granted.  It was the awareness all men had for others--that thing which whispered a warning, in the drowsiness of sleep, when someone entered the room.  In Vasher, that sense had been magnified fifty times.\nAnd now it was gone.  Sucked into the cloak and the straw person, giving them power.\nThe cloak jerked.  Vasher leaned down.  “Protect me,” he Commanded, and the cloak grew still.  He stood, throwing it back on.\nThe straw figure returned to his window.  It carried a large ring of keys.  The figure’s straw feet were stained red.  The crimson blood seemed so dull to Vasher now.\nHe took the keys.  “Thank you,” he said.  He always thanked them.  He didn’t know why, particularly considering what he did next.  “Your Breath to mine,” he commanded, touching the straw person’s chest.  The straw person immediately fell backward off the door--life draining from it--and Vasher got his Breath back.  The familiar sense of awareness returned, the knowledge of connectedness, of  fitting.  He could only take the Breath back because he’d Awakened this creature himself--indeed, Awakenings of this sort were rarely permanent.  He used his Breath like a reserve, doling it out, then recovering it.\nCompared to what he had once held, twenty-five Breaths was a laughably small number.\nHowever, compared to nothing, it seemed infinite.  He shivered in satisfaction.\nThe yells from the guard room died out.  The dungeon fell still.  He had to keep moving.\nVasher reached through the bars, using the keys to unlock his cell.  He pushed the thick door open, rushing out into the hallway, leaving the straw figure discarded on the ground.  He Sanderson/Warbreaker/12\ndidn’t walk to the guard room--and the exit beyond it--but instead turned south, penetrating deeper into the dungeon.\nThis was the most uncertain part of his plan.  Finding a tavern that was frequented by priests of the Iridescent Tones had been easy enough.  Getting into a bar fight--then striking one of those same priests--had been equally simple.  Hallandrens took their religious figures very seriously, and Vasher had earned himself not the usual imprisonment in a local jail, but a trip to the God King’s dungeons.\nKnowing the kind of men who tended to guard such dungeons, he’d had a pretty good idea that they would try to draw Nightblood.  That had given him the diversion he’d needed to get the keys.\nBut now came the unpredictable part.\nVasher stopped, Awakened cloak rustling.  It easy to locate the cell he wanted, for around it a large patch of stone had been drained of color, leaving both walls and doors a dull grey.  It was a place to imprison an Awakener, for no color meant no Awakening.  Vasher stepped up to the door, looking through the bars.  A man hung by his arms from the ceiling, naked and chained.\nHis color was vibrant to Vasher’s eyes, his skin a pure tan, his bruises brilliant splashes of blue and violet.\nThe man was gagged.  Another precaution.  In order to Awaken, the man would need three things: Breath, color, and a Command.  The Harmonics and the Hues, some called it.  The Iridescent tones, the relationship between color and sound.  A Command had to be spoken clearly and firmly in the Awakener’s native language--any stuttering, any mispronunciation, would invalidate the Awakening.  The Breath would be drawn out, but the object would be unable to act.\nSanderson/Warbreaker/13\nVasher used the prison keys to unlock the cell door, then stepped inside.  This man’s aura made colors grow brighter by sharp measure when they got close to him.  Anyone would be able to notice an aura that strong, though it was much easier for someone who had reached the First Heightening.\nIt wasn’t the strongest BioChromatic aura Vasher had ever seen--those belonged to the Returned, known as gods here in Hallandren.  Still, the prisoner’s BioChroma was very impressive and much, much stronger than Vasher’s own.  The prisoner held a lot of Breaths.\nHundreds upon hundreds of them.\nThe man swung in his bonds, studying Vasher, gagged lips bleeding from lack of water.\nVasher hesitated only briefly, then reached up and pulled the gag free.\n“You,” the prisoner whispered, coughing slightly.  “Are you here to free me?”\n“No, Vahr,” Vasher said quietly.  “I’m here to kill you.”\nVahr snorted.  Captivity hadn’t been easy on him.  When Vasher had last seen Vahr, he’d been plump.  Judging by his emaciated body, he’d been without food for some time now.  The cuts, bruises, and burn marks on his flesh were fresh.\nBoth the torture and the haunted look in Vahr’s bag-rimmed eyes bespoke a solemn truth.\nBreath could only be transferred by willing, intentional Command.  That Command could, however, be encouraged.\n“So,” Vahr croaked, “you judge me, just like everyone else.”\n“Your failed rebellion is not my concern.  I just want your Breath.”\n“You and the entire Hallandren court.”\n“Yes.  But you’re not going to give it to one of the Returned.  You’re going to give it to me.  In exchange for killing you.”\nSanderson/Warbreaker/14\n“Doesn’t seem like much of a trade.”  There was a hardness--a void of emotion--in Vahr that Vasher had not seen the last time they had parted, years before.\nOdd,  Vasher thought,  that I should finally, after all of this time, find something in the man that I can identify with. \nVasher kept a wary distance from Vahr.  Now that the man’s voice was free, he could Command.  However, he was touching nothing except for the metal chains, and metal was very difficult to Awaken.  It had never been alive, and it was far from the form of a man.  Even during the height of his power, Vasher himself had only managed to Awaken metal on a few, select occasions.  Of course, some extremely powerful Awakeners could bring objects to life that they weren’t touching, but which were in the sound of their voice.  That, however, required the Ninth Heightening.  Even Vahr didn’t have  that much Breath.  In fact, Vasher knew of only one living person who did: the God King himself.\nThat meant Vasher was probably safe.  Vahr contained a great wealth of Breath, but had nothing to Awaken.  Vasher walked around the chained man, finding it very difficult to offer any sympathy.  Vahr had earned his fate.  Yet the priests would not let him die while he held so much Breath; if he died, it would be wasted.  Gone.  Irretrievable.\nNot even the government of Hallandren--which had such strict laws about the buying and passing of Breath--could let such a treasure slip away.  They wanted it badly enough to forestall the execution of even a high-profile criminal like Vahr.  In retrospect, they would curse themselves for not leaving him better guarded.\nBut, then, Vasher had been waiting two years for an opportunity like this one.\n“Well?” Vahr asked.\n“Give me the Breath, Vahr,” Vasher said, stepping forward.\nSanderson/Warbreaker/15\nVahr snorted.  “I doubt you have the skill of the God King’s torturers, Vasher--and I’ve withstood them for two weeks now.”\n“You’d be surprised.  But that doesn’t matter.  You  are going to give me your Breath.\nYou know you have only two choices.  Give it to me, or give it to them.”\nVahr hung by his wrists, rotating slowly.  Silent.\n“You don’t have much time to consider,” Vasher said.  “Any moment now, someone is going to discover the dead guards outside.  The alarm will be raised.  I’ll leave you, you  will be tortured again, and you  will eventually break.  Then all the power you’ve gathered will go to the very people you vowed to destroy.”\nVahr stared at the floor.  Vasher let him hang for a few moments, and could see that the reality of the situation was clear to him.  Finally, Vahr looked up at Vasher.  “That. . .thing you bear.  It’s here, in the city?”\nVasher nodded.\n“The screams I heard earlier?  It caused them?”\nVasher nodded again.\n“How long will you be in T’Telir?”\n“For a time.  A year, perhaps.”\n“Will you use it against them?”\n“My goals are my own to know, Vahr.  Will you take my deal or not?  Quick death in exchange for those Breaths.  I promise you this.  Your enemies will  not have them.”\nVahr grew quiet.  “It’s yours,” he finally whispered.\nVasher reached over, resting his hand on Vahr’s forehead--careful not to let any part of his clothing touch the man’s skin, lest Vahr draw forth color for Awakening.\nSanderson/Warbreaker/16\nVahr didn’t move.  He looked numb.  Then, just as Vasher began to worry that the prisoner had changed his mind, Vahr Breathed.  The color drained from him.  The beautiful Iridescence, the aura that had made him look majestic despite his wounds and chains.  It flowed from his mouth, hanging in the air, shimmering like mist.  Vasher drew it in, closing his eyes.\n“My life to yours,” Vahr Commanded, a hint of despair in his voice.  “My Breath become yours.”\nThe Breath flooded into Vasher, and everything became vibrant.  His brown cloak now seemed deep and rich in color.  The blood on the floor was intensely red, as if aflame.   Even Vahr’s skin seemed a masterpiece of color, the surface marked by deep black hairs, blue bruises, and sharp red cuts.  It had been years since Vasher had felt such. . .  life.\nHe gasped, falling to his knees as it overwhelmed him, and he had to drop a hand to the stone floor to keep himself from toppling over.  How did I live without this? \nHe knew that his senses hadn’t actually improved, yet he felt so much more alert.  More aware of the beauty of sensation.  When he touched the stone floor, he marveled at its roughness.\nAnd the sound of wind passing through the thin dungeon window up above.  Had it always been that melodic?  How could he not have noticed?\n“Keep your part of the bargain,” Vahr said.  Vasher noted the tones in his voice, the beauty of each one, how close they were to harmonics.  Vasher had gained perfect pitch.  A gift for anyone who reached the Second Heightening.  It would be good to have that again.\nVasher could, of course, have up to the Fifth Heightening at any time, if he wished.  That would require certain sacrifices he wasn’t willing to make.  And so, he forced himself to do it the old fashioned way, by gathering Breaths from people like Vahr.\nSanderson/Warbreaker/17\nVasher stood, then pulled out the colorless scarf he had used earlier.  He tossed it over Vahr’s shoulder, then Breathed.\nHe didn’t bother making the scarf have human shape, didn’t need to use a bit of his hair or skin for a focus--though he did have to draw the color from his shirt.\nVasher met Vahr’s resigned eyes.\n“Strangle things,” Vasher Commanded, fingers touching the quivering scarf.\nIt twisted immediately, pulling away a large--yet now inconsequential--amount of Breath.\nThe scarf quickly wrapped around Vahr’s neck, tightening, choking him.  Vahr didn’t struggle or gasp, he simply watched Vasher with hatred until his eyes bulged and he died.\nHatred.  Vasher had known enough of that in his time.  He quietly reached up and recovered his Breath from the scarf, then left Vahr dangling in his cell.  Vasher passed quietly though the prison, marveling at the color of the woods and the stones.  After a few moments of walking, he noticed a new color in the hallway.  Red.\nHe stepped around the pool of blood--which was seeping down the inclined dungeon floor--and moved into the guard room.  The three guards lay dead.  One of them sat in a chair.\nNightblood, still mostly sheathed, had been rammed through the man’s chest.  About an inch of a dark black blade was visible beneath the silver sheath.\nVasher carefully slid the weapon fully back into its sheath.  He did up the clasp.\nI did very well today,  a voice said in his mind.\nVasher didn’t respond to the sword.\nI killed them all,  Nightblood continued.  Aren’t you proud of me? \nVasher picked up the weapon, accustomed to its unusual weight, and carried it in one hand.  He recovered his duffle and slung it over his shoulder.\nSanderson/Warbreaker/18\nI knew you’d be impressed,  Nightblood said, sounding satisfied.\nSanderson/Warbreaker/19\n\n\n\n'

In [59]:
get_feature_vectors(textt)

(['Vasher',
  'Vahr',
  'Breath',
  'Command',
  'Awaken',
  'Nightblood',
  'Hallandren',
  'Breaths',
  'BioChromatic',
  'Iridescent Tones',
  'Returned Gods',
  'Quick',
  'Second Heightening',
  'Blues',
  'Fetch',
  'Awakener',
  'BioChroma',
  'Captivity',
  'Odd',
  'Hues',
  'Vahr Breathed',
  'Vasher Breathed',
  'Gone',
  'BioChromatic Breath',
  'Ninth Heightening',
  'Breaths Vasher',
  'Society',
  'Bright',
  'Darker',
  'Lifeless',
  'Hatred',
  'Aren',
  'Fifth Heightening',
  'Returned',
  'Awakened',
  'Colorless',
  'Anyone',
  'Silent',
  'Red'],
 array([[  0.        ,   0.        ,   0.        , ...,  36.84210526,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,  14.28571429,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   8.33333333,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ..., 100.        ,
           0.      

In [49]:
warbreaker_characters = run_classifier(extract_texts_and_characters(warbreaker)[0], 
                       classifier=cls,)
ann_warbreaker = [(char, datum['text'][1:125]) for char,datum in zip(warbreaker_characters, warbreaker)]
ann_warbreaker

[('WARBREAKER',
  '\nIntroduction to WARBREAKER\nWelcome!  My name is Brandon Sanderson.  Before anything else, I’d like to thank you for your in'),
 ('Vasher',
  '\nPrologue\nIt’s funny,  Vasher thought,  how many things begin with me getting thrown into prison. \nThe guards laughed to one'),
 ('Dedelin',
  '\nChapter One\nThere were great advantages to being unimportant.\nTrue, by many people’s standards, Siri wasn’t ‘unimportant.’ '),
 ('Vivenna',
  '\nChapter Two\nSiri sat, stunned, in a rattling carriage, her homeland growing more and more distant with each bump and shake.'),
 ('Lightsong',
  '\nChapter Three\nLightsong didn’t remember dying.\nHis priests, however, assured him that his death had been extremely inspirin'),
 ('Siri',
  '\nChapter Four\nSiri’s carriage rolled to a stop outside of T’Telir, capital of Hallandren.  She stared out the window and rea'),
 ('Vasher',
  '\nChapter Five\nThis will complicate things,  Vasher thought, standing in the shadows atop the wall that 

In [ ]:
for imp, name in zip(classifier.feature_importances_, FeatureVec().keys()):
    print(name, "\t", imp )